In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

video_path = '스쿼트.mp4'
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('%s_output.mp4' % (video_path.split('.')[0]), fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

#허리 틀어짐 카운트 변수
left_hip_count = 0
right_hip_count =0 
total_hip_err = 0

is_first = True # 어드레스 시 첫 프레임을 받아오기 위한 플래그

#어드레스 시 첫 프레임의 좌표를 저장할 변수
# first_center_x, first_center_y = None, None
first_left_hip_x, first_right_hip_x = None,None
first_shoulder_x1,first_shoulder_x2,first_shoulder_y1,first_shoulder_y2 =None, None,None, None
first_left_knee_x,first_left_knee_y,first_right_knee_x,first_right_knee_y = None,None,None,None
first_right_heel, first_right_foot_index, first_right_angle=None,None,None
hip_cnt = 0
stage = None

with mp_pose.Pose(
    min_detection_confidence=0.5,
    enable_segmentation=True,
    min_tracking_confidence=0.5) as pose:
    
                    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("카메라를 찾을 수 없습니다.")
# 동영상을 불러올 경우는 'continue' 대신 'break'를 사용합니다.
            break
        image_h, image_w, _ = image.shape
        img_result = image.copy()

# 필요에 따라 성능 향상을 위해 이미지 작성을 불가능함으로 기본 설정합니다.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

# 포즈 주석을 이미지 위에 그립니다.
        img_result.flags.writeable = True
        img_result = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            img_result,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


        if results.pose_landmarks:
            # https://google.github.io/mediapipe/solutions/pose.html#pose-landmark-model-blazepose-ghum-3d
            landmark = results.pose_landmarks.landmark

#힙 좌표
            left_hip_x = int(landmark[mp_pose.PoseLandmark.LEFT_HIP].x * image_w)
            left_hip_y = int(landmark[mp_pose.PoseLandmark.LEFT_HIP].y * image_h)

            right_hip_x = int(landmark[mp_pose.PoseLandmark.RIGHT_HIP].x * image_w)
            right_hip_y = int(landmark[mp_pose.PoseLandmark.RIGHT_HIP].y * image_h)
            
#무릎 좌표
            left_knee_x = int(landmark[mp_pose.PoseLandmark.LEFT_KNEE].x * image_w)
            left_knee_y = int(landmark[mp_pose.PoseLandmark.LEFT_KNEE].y * image_h)
    
            right_knee_x = int(landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x * image_w)
            right_knee_y = int(landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x * image_h)
            
#어깨 좌표
            left_shoulder_x = int(landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_w)
            left_shoulder_y = int(landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_h)
    
            right_shoulder_x = int(landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_w)
            right_shoulder_y = int(landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_h)
            
            
#각도                     
            hip = [landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].y] #엉덩이 
    
            center_x = (landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].x + landmark[mp_pose.PoseLandmark.LEFT_HIP.value].x)/2
            center_y = (landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].y + landmark[mp_pose.PoseLandmark.LEFT_HIP.value].y)/2
            center =  [center_x,center_y]       #엉덩이 가상의 점 중앙
    
            shoulder = [landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y] #어깨 
            
            right_heel = [landmark[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmark[mp_pose.PoseLandmark.RIGHT_HEEL.value].y] #발뒷꿈치 
            right_foot_index = [landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y] #오른쪽 발가락
            right_angle = [landmark[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y] #발의 각도를 구하기 위한 가상의 점

            knee = [landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].y] #무름 
            right_angle2= [center_x,landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]  #무릎의 각도를 구하기 위한 가상의 점
            
            ankle = [landmark[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmark[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y] #발목 각도
            
        
            if is_first: # 어드레스 시 첫 프레임의 골판 좌표 저장
                first_left_hip_x = left_hip_x
                first_left_hip_y = left_hip_y

                first_right_hip_x = right_hip_x
                first_right_hip_y = right_hip_y

                first_left_shoulder_x = left_shoulder_x
                first_left_shoulder_y = left_shoulder_y

                first_right_shoulder_x = right_shoulder_x
                first_right_shoulder_y = right_shoulder_y

                first_left_knee_x = left_knee_x
                first_left_knee_y = left_knee_y

                first_right_knee_x = right_knee_x
                first_right_knee_y = right_knee_y
                
                first_right_heel = right_heel
                first_right_foot_index = right_foot_index
                first_right_angle = right_angle
                #가상의 점, 발 뒷꿈치, 발가락 끝
                angle2 = calculate_angle(first_right_angle,first_right_heel,first_right_foot_index) #발의 각도
                is_first = False
            else:
                angle = calculate_angle(shoulder,hip,knee) #엉덩이 각도
                angle3 = calculate_angle(right_angle2,center,knee) #무릎이 얼마나 벌려져있는지 

                #엉덩이
                cv2.circle(img_result, center = (first_left_hip_x,left_hip_y ), radius = 10, color=(0, 255, 255),thickness=2)
                cv2.circle(img_result, center = (first_right_hip_x,right_hip_y ), radius = 10 , color=(0, 255, 255),thickness=2)

                color = (0, 255, 0) # 초록색

                # 골반 원래 위치보다  벗어난 경우
                if left_hip_x<first_left_hip_x-5:
                    color = (0, 0, 255) # 빨간
                    left_hip_count +=1
                    total_hip_err+=1
                elif first_right_hip_x+5<right_hip_x:
                    color = (0, 0, 255) # 빨간
                    right_hip_count +=1
                    total_hip_err+=1
                cv2.circle(img_result, center = (left_hip_x,left_hip_y ), radius = 5, color=color,thickness=2)
                cv2.circle(img_result, center = (right_hip_x,right_hip_y ), radius = 5 , color=color,thickness=2)
                    
                #어깨
                cv2.circle(img_result, center = (first_left_shoulder_x,left_shoulder_y ), radius = 10, color=(0, 255, 255),thickness=2)
                cv2.circle(img_result, center = (first_right_shoulder_x,right_shoulder_y ), radius = 10 , color=(0, 255, 255),thickness=2)                
                
                #어깨 틀어질경우
                if left_shoulder_x < first_left_shoulder_x-5: 
                    color = (0,0,255) #빨간색
                   
                elif first_right_shoulder_x + 5 < right_shoulder_x:
                    color = (0,0,255) #빨간색
                 

                cv2.circle(img_result, center = (left_shoulder_x,left_shoulder_y ), radius = 5 , color=color,thickness=2)                
                cv2.circle(img_result, center = (right_shoulder_x,right_shoulder_y ), radius = 5 , color=color,thickness=2)
                
                #동작 수행
                #첫 발의 각도
                if angle2<30 or angle2>=46:
                    cv2.putText(img_result,"right_heel_angel : "+str(int(angle2)),(50,100),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
                
                if angle < 140:
                    stage = "down"
                    #if angle3<100: 
#                         cv2.putText(img_result,"얼마만큼 : "+str(int(angle3)),(50,150),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)                
                
                elif angle < 170 and stage =='down':
                    stage="up"
                    hip_cnt += 1
                cv2.putText(img_result,"right_heel_angel : "+str(int(angle2)),(50,100),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)    
                cv2.putText(img_result,"hip_angle : "+str(int(angle)),(50,200),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)                
                cv2.putText(img_result,"heel_angle : "+str(int(angle3)),(50,150),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)                
                cv2.putText(img_result,"count : " + str(hip_cnt),(50,50),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
                # cv2.putText(img_result,"left_hip_err : " + str(left_hip_count/total_hip_err*100) + "right_hip_err"+ str(right_hip_count/total_hip_err*100),(50,50),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                cv2.imshow('MediaPipe Pose', img_result)
                out.write(img_result)
                if cv2.waitKey(1)==ord("w"):
                        time.sleep(3)
                if cv2.waitKey(1)==ord("q"):
                    break

cap.release()
cv2.destroyAllWindows()
left_hip_per_err = left_hip_count/total_hip_err*100
right_hip_per_err = right_hip_count/total_hip_err*100
print("left_hip_per_err : " , left_hip_per_err)
print("right_hip_per_err : " , right_hip_per_err)


left_hip_per_err :  100.0
right_hip_per_err :  0.0
